# Imports

## System imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json
import re

import seaborn as sns
import os

## Data imports

v1: predbezne vysledky po errate, ulozene na SharePointe OSN. Este sa budu updatovat

In [3]:
all_inpatients_file_path = r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\osn_vsetka_starostlivost_2023_ms_v2024.2.3.csv"

In [4]:
all_inpatients = pd.read_csv(all_inpatients_file_path, dtype=str, delimiter=";")

In [5]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,uroven_ms
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,1~3
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,1~3
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,1~3
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,1~2
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,3


In [6]:
all_inpatients.dtypes

id_hp                 object
vek_roky              object
vek_dni               object
hmotnost              object
upv                   object
diagnozy              object
vykony                object
drg                   object
erv                   object
typ_starostlivosti    object
diagnozy_z_01         object
vykony_z_01           object
obdobie_od            object
obdobie_do            object
datum_od              object
datum_do              object
osetrovacia_doba      object
id_poistenca          object
datum_narodenia       object
kod_pobytu            object
kod_zp                object
pzs_12                object
IDENTIFZAR            object
pzs_8                 object
pzs_6                 object
id_hp_pzs             object
ms                    object
uroven_ms             object
dtype: object

In [7]:
all_inpatients.shape

(1178065, 28)

## Hospitals information

In [8]:
pzs_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\PZS_zakl_info_GPS_bezprepojeni.xlsx", dtype=str)

In [9]:
pzs_gps.rename(columns={'IDENTIFZAR': 'pzs_8', 'SIDZAR_KOD_OBEC': 'KOD_OBEC_PZS', 'SIDZAR_KOD_OKRES_PZS': 'KOD_OKRES_PZS', 'lat_pzs_ifexists': 'lat_pzs', 'lng_pzs_ifexists': 'lng_pzs', 'NAZZAR': 'NAZOV_PZS', 'OSN_siet' : 'OSN_siet', 'OSN_level' : 'OSN_level', 'typ_nemocnice' : 'typ_nemocnice'}, inplace=True)

In [10]:
pzs_gps.head(2)

,pzs_8,PZS_short,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OBEC_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,NAZZAR_povodne,lat_pzs_ifexists_povodne,lng_pzs_ifexists.1,SIDZAR_KOD_OBEC_povodne,SIDZAR_KOD_OKRES_PZS_povodne
0,P5848001,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",nie,mimo siet OSN,mimo siet OSN,501760,201,47.8001114823,17.6645174625,"Ministerstvo vnútra Slovenskej republiky, všeo...",47.8001114823,17.6645174625,501760,201
1,P5848002,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",nie,mimo siet OSN,mimo siet OSN,520004,702,48.9336706585,21.9065813542,"Ministerstvo vnútra Slovenskej republiky, všeo...",48.9336706585,21.9065813542,520004,702


## Patient information

In [11]:
pacient_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\obce_gps_simple.xlsx")

In [12]:
pacient_gps = pacient_gps.rename(columns={'RESIDENCY': 'kod_pobytu'})

In [13]:
pacient_gps.head()

,kod_pobytu,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,okresne_mesto_zuj_pacient,okresne_mesto_pacient_nazov,lat_okres_pacient,lng_okres_pacient,residency_count
0,501280,Nesvady,401,Komárno,501026,Komárno,47.765799,18.119551,1
1,504025,Sala,405,Šaľa,504025,Šaľa,48.156715,17.875971,1
2,504092,Trnovec nad Vahom,405,Šaľa,504025,Šaľa,48.156715,17.875971,1
3,555509,Zalesie,108,Senec,508217,Senec,48.219813,17.396061,1
4,504131,Velke Ulany,202,Galanta,503665,Galanta,48.189953,17.725027,1


In [14]:
pacient_gps.dtypes

kod_pobytu                      object
Obec_pacient_nazov              object
id_okres_pacient                object
Okres_pacient_nazov             object
okresne_mesto_zuj_pacient       object
okresne_mesto_pacient_nazov     object
lat_okres_pacient              float64
lng_okres_pacient              float64
residency_count                  int64
dtype: object

## MS names and levels

Update 31.7. po poslednej Oliverovej zmene zdroja (pridanie urovne MS). Uz to tym padom nie je potrebne priradovat. Tento zdroj potrebujem IBA KVOLI NAZVOM MS

Tento zoznam treba "oholit" na uplne jednoduchu tabulku bez duplicit, inak to robi problemy a duplikuje riadky. To znamena, ze treba dat prec aj zdielane sluzby. Upravoval som este samotny excel.

In [15]:
ms_names_levels = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\02_Zoznam-medicinskych-sluzieb 1 adjusted.xlsx")

In [16]:
ms_names_levels.head()

,Číslo programu,Kód medicínskej služby,ms_code,Zdieľaná medicínska služba,ms_name,typ,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18
0,1,S01-04,S01-04,NaN,Bezvedomie a zabezpečenie DC vyžadujúce úzko š...,V,IV,NaN,NaN,NaN,NaN
1,1,S01-03,S01-03,NaN,Podpora dýchania vyžadujúca úzko špecializovan...,V,IV,NaN,NaN,NaN,NaN
2,1,S01-02,S01-02,NaN,Podpora krvného obehu vyžadujúca úzko špeciali...,V,IV,NaN,NaN,NaN,NaN
3,1,S01-01,S01-01,NaN,Kritické poruchy vnútorného prostredia vyžaduj...,V,IV,NaN,NaN,NaN,NaN
4,1,S01-14,S01-14,NaN,Komplexná intenzívna starostlivosť po transpla...,D,III,NaN,NaN,NaN,NaN


## List of medical programmes

In [17]:
medical_programmes_list = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\Medicinske_programy_zoznam.xlsx")

In [18]:
medical_programmes_list.head()

,Oznacenie_prislusnych_MS_SXX,Cislo_programu,Nazov_programu,Cislo_a_nazov_programu
0,S01,1,Program anestéziológie a intenzívnej medicíny,1 Program anestéziológie a intenzívnej medicíny
1,S02,2,Traumatologický program,2 Traumatologický program
2,S03,3,Neurochirurgický program,3 Neurochirurgický program
3,S04,4,Oftalmologický program,4 Oftalmologický program
4,S05,5,Otorinolaryngologický program,5 Otorinolaryngologický program


# Data processing

## Choose first MS 

In [19]:
all_inpatients['ms_code'] = all_inpatients['ms'].str.slice(0,6)

In [20]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,uroven_ms,ms_code
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,1~3,S05-34
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,1~3,S05-34
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,1~3,S05-34
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,1~2,S53-33
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,3,S53-29



## Merge MS names

In [21]:
ms_names_levels_NamesOnly = ms_names_levels[['ms_code', 'ms_name']]

In [22]:
ms_names_levels_NamesOnly.head()

,ms_code,ms_name
0,S01-04,Bezvedomie a zabezpečenie DC vyžadujúce úzko š...
1,S01-03,Podpora dýchania vyžadujúca úzko špecializovan...
2,S01-02,Podpora krvného obehu vyžadujúca úzko špeciali...
3,S01-01,Kritické poruchy vnútorného prostredia vyžaduj...
4,S01-14,Komplexná intenzívna starostlivosť po transpla...


In [23]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_NamesOnly, on='ms_code', how='left')

## UPDATE 31.7.2024 - NEPOTREBNE: Based on age, assign ms level

31.7.2024: MS level pre kazdu MS pridal Oliver v zdroji dat. Nemusim priradovat tu. Rovnako, ako ma oliver cez znak spolu viacere MS, ma aj pre kazdu uroven. Podobne ako ponechavam iba jednu (prvu) MS, ponecham len jednu (prvu) uroven.

In [24]:
#ms_names_levels_AllLevels = ms_names_levels[['ms_code', 'level_dospely', 'level_do1r', 'level_1az6', 'level_7az15', 'level_16az18']]

In [25]:
#ms_names_levels_AllLevels.head()

In [26]:
#all_inpatients = pd.merge(all_inpatients, ms_names_levels_AllLevels, on='ms_code', how='left')

In [27]:
def determine_ms_level(row):
    if row['vek_roky'] <= 1:
        return row['level_do1r']
    elif row['vek_roky'] <= 6:
        return row['level_1az6']
    elif row['vek_roky'] <= 15:
        return row['level_7az15']
    elif row['vek_roky'] <= 18:
        return row['level_16az18']    
    else:
        return np.nan

In [28]:
#nan_count_vek_roky = all_inpatients['vek_roky'].isna().sum()
#print(nan_count_vek_roky)

kedze su len 2 NaN values, dam ich prec, inak by bolo treba napisat funkciu, aby sa NaNs dali napr. na minus 1

In [29]:
#all_inpatients = all_inpatients.dropna(subset=['vek_roky'])

In [30]:
#all_inpatients['vek_roky'] = all_inpatients['vek_roky'].astype(int)

In [31]:
#all_inpatients['ms_level'] = all_inpatients.apply(determine_ms_level, axis=1)

In [32]:
#all_inpatients['ms_level'] = all_inpatients['ms_level'].fillna(all_inpatients['level_dospely'])

In [33]:
#nan_count_ms_level = all_inpatients['ms_level'].isna().sum()
#print(nan_count_ms_level)

In [34]:
#filtered_df = all_inpatients.loc[all_inpatients['id_hp'] == "24283_23005692"]

In [35]:
#filtered_df

zostava mi stale okolo 100 NaNs, mozno chyba vo vykazovani, lebo take pripady nemaju level vo svojej kategorii podla veku a ani pre dospeleho. 

In [36]:
#later delete 
#all_inpatients = all_inpatients.drop(columns=['level_dospely', 'level_do1r', 'level_1az6', 'level_7az15', 'level_16az18'])

In [37]:
#all_inpatients.shape

## Assign just one MS level

Predtym som to tu musel priradovat, avsak po update 31.7. to mam uz v zdroji od Olivera. Ten ma niekedy viac MS a teda aj viac urovni pre kazdu. Podobne ako nechavam iba jednu (prvu) uroven MS, nechavam aj len jednu (prvu) uroven danej (prvej) MS.

In [38]:
all_inpatients['ms_level'] = all_inpatients['uroven_ms'].str.slice(0,1)

In [39]:
# mozem dropnut povodnu Oliverovu "uroven_ms", nepotrebujem na nic dalsie
all_inpatients = all_inpatients.drop(columns=['uroven_ms'])

In [40]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,ms_level
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,1
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,1
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,1
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,S53-33,Tonzilektómia alebo adenoidektómia,1
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,S53-29,"Poruchy rovnováhy, strata sluchu alebo tinnitus",3


## Adding type of MS

In [41]:
ms_names_levels_msTypes = ms_names_levels[['ms_code', 'typ']]

In [42]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_msTypes, on='ms_code', how='left')

In [43]:
all_inpatients.shape

(1178065, 31)

## Adding PZS (hospital) GPS and additional information

In [44]:
all_inpatients = all_inpatients.merge(pzs_gps[['pzs_8','NAZOV_PZS', 'OSN_siet', 'OSN_level', 'typ_nemocnice', 'KOD_OKRES_PZS', 'lat_pzs', 'lng_pzs']], on='pzs_8', how='left')

## Adding patient GPS and disctrict

In [45]:
all_inpatients['kod_pobytu'] = all_inpatients['kod_pobytu'].astype(str)
pacient_gps['kod_pobytu'] = pacient_gps['kod_pobytu'].astype(str)

In [46]:
all_inpatients = all_inpatients.merge(pacient_gps[['kod_pobytu','Obec_pacient_nazov', 'id_okres_pacient', 'Okres_pacient_nazov', 'lat_okres_pacient', 'lng_okres_pacient']], on='kod_pobytu', how='left')

In [47]:
all_inpatients['id_okres_pacient'].isna().sum()

np.int64(1771)

## Add main programme number and name

In [48]:
#adding main programme name
all_inpatients = pd.merge(all_inpatients,
    medical_programmes_list,
    left_on=all_inpatients['ms_code'].str[:3],
    right_on=medical_programmes_list['Oznacenie_prislusnych_MS_SXX'].str[:3],
    how='left'
    )

In [49]:
#delete unnecessary columns after adding main programme name
all_inpatients = all_inpatients.drop(columns=['key_0', 'Oznacenie_prislusnych_MS_SXX', 'Cislo_programu', 'Nazov_programu'])

## Preliminary output

In [50]:
all_inpatients.shape

(1178065, 44)

In [51]:
all_inpatients['id_hp'].nunique()

1178065


v 2022 som mal menej unikatnych 'id_hp' ako celkovo riadkov, cize boli asi nejake duplicity. Tu nie su, co je super. 

In [52]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,ms_level,typ,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,Cislo_a_nazov_programu
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,1,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Michalovce,807,Michalovce,48.754228,21.910748,5 Otorinolaryngologický program
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,1,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Okruhle,712,Svidník,49.306993,21.569144,5 Otorinolaryngologický program
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,1,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Vysna Sitnica,702,Humenné,48.933671,21.906581,5 Otorinolaryngologický program
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,S53-33,Tonzilektómia alebo adenoidektómia,1,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Certizne,705,Medzilaborce,49.275750,21.905336,53 Otorinolaryngologický program pre deti
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,S53-29,"Poruchy rovnováhy, strata sluchu alebo tinnitus",3,D,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Snina,709,Snina,48.987886,22.158990,53 Otorinolaryngologický program pre deti


In [ ]:
## EXPORT
#all_inpatients.to_csv("all_inpatients.csv")

## Adding age parametrical values

In [54]:
all_inpatients['vek_roky'] = all_inpatients['vek_roky'].astype('Int64')

In [55]:
# Define age cohorts
bins = [-1, 14, 24, 34, 44, 54, 64, 74, float('inf')]
labels = ['under 15', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75+']

# Add new column to DataFrame with cohorts
all_inpatients['age_cohort'] = pd.cut(all_inpatients['vek_roky'], bins=bins, labels=labels, right=True)


In [56]:
all_inpatients['age_cohort'].isna().sum()

np.int64(2)

In [57]:
na_rows = all_inpatients[all_inpatients['age_cohort'].isna()]

In [58]:
na_rows.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,ms_level,typ,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,Cislo_a_nazov_programu,age_cohort
786519,275825,<NA>,NaN,NaN,NaN,i214~i200,NaN,NaN,NaN,UH,i214~i200,NaN,2023-12,2023-12,2023-12-27,2023-12-28,1,9866056,NaN,nan,24,P68335278101,P6833501,P6833501,P68335,NaN,S99-99,S99-99,Medicínska služba pre nezaradené hospitalizačn...,NaN,NaN,"CINRE s. r. o., všeobecná nemocnica, Bratislava",ano,4,Špecializovaná nemocnica,105,48.101866,17.109484,NaN,NaN,NaN,NaN,NaN,99 Program pre nezaradené hospitalizačné prípady,NaN
836863,x46923,<NA>,NaN,NaN,NaN,n47,5p501&Z&202311xx,NaN,NaN,JZS,n47,5p501&Z&202311xx,2023-11,2023-11,2023-11-10,NaN,1,7336932,NaN,nan,24,P59790012401,P5979001,P5979001,P59790,NaN,S99-99,S99-99,Medicínska služba pre nezaradené hospitalizačn...,NaN,NaN,"Euromedix, a.s., poliklinika, Bratislava",nie,mimo siet OSN,mimo siet OSN,105,48.11170995,17.1005310373,NaN,NaN,NaN,NaN,NaN,99 Program pre nezaradené hospitalizačné prípady,NaN


In [59]:
age_inspection_rows = all_inpatients[all_inpatients['vek_roky'] == 25]

In [60]:
age_inspection_rows.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,ms_level,typ,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,Cislo_a_nazov_programu,age_cohort
470,36454_23000878,25,NaN,NaN,NaN,o800~o096~o990~z370,NaN,o60d,0.573,DRG,o800,NaN,2023-01,2023-01,2023-01-15,2023-01-19,4,7917802,1997-05-03,503193,24,P81095009101,P8109501,P8109501,P81095,23000878,S11-15,S11-15,Pôrod,2,D,"Fakultná nemocnica s poliklinikou Nové Zámky, ...",ano,2,Všeobecná nemocnica,404,47.98997,18.153291,Hul,404,Nové Zámky,47.989298,18.170745,11 Pôrodnícky program,25-34
471,36454_23000881,25,NaN,NaN,NaN,o800~o096~z370,NaN,o60d,0.573,DRG,o800~o808,NaN,2023-01,2023-01,2023-01-15,2023-01-20,5,9481067,1997-11-29,556785,24,P81095009101,P8109501,P8109501,P81095,23000881,S11-15,S11-15,Pôrod,2,D,"Fakultná nemocnica s poliklinikou Nové Zámky, ...",ano,2,Všeobecná nemocnica,404,47.98997,18.153291,Male Chyndice,403,Nitra,48.309149,18.079528,11 Pôrodnícky program,25-34
472,36454_23001335,25,NaN,NaN,NaN,o800~z370~o096,NaN,o60d,0.573,DRG,o800,NaN,2023-01,2023-01,2023-01-21,2023-01-26,5,4035301,1997-11-30,504025,24,P81095009101,P8109501,P8109501,P81095,23001335,S11-15,S11-15,Pôrod,2,D,"Fakultná nemocnica s poliklinikou Nové Zámky, ...",ano,2,Všeobecná nemocnica,404,47.98997,18.153291,Sala,405,Šaľa,48.156715,17.875971,11 Pôrodnícky program,25-34
600,36454_23001713,25,NaN,NaN,NaN,k8080~o266~o094~o996,NaN,h64z,0.2394,DRG,k8080,NaN,2023-01,2023-01,2023-01-26,2023-01-27,1,3443741,1997-02-28,501417,24,P81095010101,P8109501,P8109501,P81095,23001713,S28-18,S28-18,Ochorenia žlčníka a žlčovodov,2,D,"Fakultná nemocnica s poliklinikou Nové Zámky, ...",ano,2,Všeobecná nemocnica,404,47.98997,18.153291,Zemianska Olca,401,Komárno,47.765799,18.119551,28 Program gastroenterológie a hepatológie,25-34
803,36454_23001707,25,NaN,NaN,NaN,s826~s8282,5t12br&P&20230129~32108&Z&20230126,i13e,1.2137,DRG,s826,5t12br&Z&202301xx,2023-01,2023-01,2023-01-26,2023-01-31,5,3780410,1997-05-09,501140,24,P81095013101,P8109501,P8109501,P81095,23001707,S02-36~S02-63,S02-36,Osteosyntéza jednoduchej zlomeniny,2,V,"Fakultná nemocnica s poliklinikou Nové Zámky, ...",ano,2,Všeobecná nemocnica,404,47.98997,18.153291,Hurbanovo,401,Komárno,47.765799,18.119551,2 Traumatologický program,25-34


## Adding "kraj" parameter

Tu mi staci osekat okres na prvy znak, co je cislo kraja

In [61]:
all_inpatients['id_kraj_pacient'] = all_inpatients['id_okres_pacient'].astype(str).str[0]


In [62]:
all_inpatients['id_kraj_pacient'].isna().sum()

np.int64(0)

# Outputs

## All inpatients aggregated by hospital, patient district and medical service

In [63]:
all_inpatients_cut = all_inpatients[['id_kraj_pacient', 'age_cohort', 'ms_level', 'ms_code', 'ms_name', 'Cislo_a_nazov_programu', 'id_hp']]

In [64]:
all_inpatients_cut.head()

,id_kraj_pacient,age_cohort,ms_level,ms_code,ms_name,Cislo_a_nazov_programu,id_hp
0,8,15-24,1,S05-34,Tonzilektómia alebo adenoidektómia,5 Otorinolaryngologický program,35380_23000079
1,7,15-24,1,S05-34,Tonzilektómia alebo adenoidektómia,5 Otorinolaryngologický program,35380_23000214
2,7,15-24,1,S05-34,Tonzilektómia alebo adenoidektómia,5 Otorinolaryngologický program,35380_23000126
3,7,15-24,1,S53-33,Tonzilektómia alebo adenoidektómia,53 Otorinolaryngologický program pre deti,35380_23000083
4,7,15-24,3,S53-29,"Poruchy rovnováhy, strata sluchu alebo tinnitus",53 Otorinolaryngologický program pre deti,35380_23000159


In [65]:
all_inpatients_cut.shape

(1178065, 7)

In [68]:
all_inpatients_pivot = (
    all_inpatients_cut.groupby(['id_kraj_pacient','age_cohort', 'ms_code'])
    .agg({'id_hp': 'nunique'})
    .reset_index()
    .rename(columns={'id_hp': 'count_of_unique_id_hp'})
)

C:\Users\hunakj\AppData\Local\Temp\ipykernel_20088\4218790429.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  all_inpatients_cut.groupby(['id_kraj_pacient','age_cohort', 'ms_code'])


In [69]:
all_inpatients_pivot.shape

(159152, 4)

(159152, 4) shape pred upravou

TO DO: tuto prezistit (spytat sa napr. Olivera) preco to tak divne zgrupuje, ze mi to spravi kombinaciu pre vsetky typy hoci ten pocet nqunique je tam 0

id pacienta nie je unikatne, id pacienta a kod ZP je unikatny
stane sa ze 2 poistovne maju id poistenca rovnake

In [ ]:
all_inpatients_pivot = all_inpatients_pivot.sort_values(by=[ 'patient_count_By_hospital_PatientDistrict_MSlevels'], ascending=[ False])

In [ ]:
all_inpatients_pivot.to_csv("all_inpatients_pivot.csv", encoding='utf_8_sig')